<a href="https://colab.research.google.com/github/supuni9622/Test-Google-Colab/blob/main/Lyrics_Classification_Ensemble_methods_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
cd /content/drive/MyDrive/4th Year Research/M3/Test Implementation/Test-Google-Colab/TestV2

/content/drive/MyDrive/4th Year Research/M3/Test Implementation/Test-Google-Colab/TestV2


In [3]:
dataset = pd.read_csv('Lyrical_data_2.csv',encoding='cp1252')

In [5]:
dataset.dropna(how='all', axis=1, inplace=True)

In [6]:
dataset.head()

,Song_ID,Song_Title,Chorus,Emotion
0,1,Diana,"Diana,\rLet me be the one to light a fire insi...",Inspiration
1,2,Don't forget where you belong,Don't forget where you belong\rHome\rDon't for...,Inspiration
2,3,Strong,"I'm sorry if I say, ""I need you""\nBut I don't ...",Inspiration
3,4,Happily,I don't care what people say when we're togeth...,Sad
4,5,Better than words,Better than words\rMore than a feeling Crazy i...,Love


In [8]:
lyrics2 = dataset['Chorus']
lyrics2

0      Diana,\rLet me be the one to light a fire insi...
1      Don't forget where you belong\rHome\rDon't for...
2      I'm sorry if I say, "I need you"\nBut I don't ...
3      I don't care what people say when we're togeth...
4      Better than words\rMore than a feeling Crazy i...
                             ...                        
276    Anyway I'm just gonna sing and play\nUm, and I...
277    Heart beats fast\rColors and promises\rHow to ...
278    It's a pity\rLow down and dirty shame\rLord, i...
279    I went down to the river\rSat there all night ...
280    You think that you have left me behind\rAnd th...
Name: Chorus, Length: 281, dtype: object

Checking the empty rows in dataset

In [9]:
print (dataset.isnull().sum())

Song_ID       0
Song_Title    0
Chorus        1
Emotion       2
dtype: int64


In [10]:
modifiedDF = dataset.dropna()
modifiedDF

,Song_ID,Song_Title,Chorus,Emotion
0,1,Diana,"Diana,\rLet me be the one to light a fire insi...",Inspiration
1,2,Don't forget where you belong,Don't forget where you belong\rHome\rDon't for...,Inspiration
2,3,Strong,"I'm sorry if I say, ""I need you""\nBut I don't ...",Inspiration
3,4,Happily,I don't care what people say when we're togeth...,Sad
4,5,Better than words,Better than words\rMore than a feeling Crazy i...,Love
...,...,...,...,...
276,277,Love me like you do,"Anyway I'm just gonna sing and play\nUm, and I...",Love
277,278,A thousand years,Heart beats fast\rColors and promises\rHow to ...,Love
278,279,Money Is The Name of The Game,"It's a pity\rLow down and dirty shame\rLord, i...",Angry
279,280,Blues In My Bottle,I went down to the river\rSat there all night ...,Sad


In [11]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 34):
  Lyrics = re.sub('[^a-zA-Z]', ' ', modifiedDF['Chorus'][i])
  Lyrics = Lyrics.lower()
  Lyrics = Lyrics.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  Lyrics = [ps.stem(word) for word in Lyrics if not word in set(all_stopwords)]
  Lyrics = ' '.join(Lyrics)
  corpus.append(Lyrics)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Creating the bag of words modal

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = modifiedDF.iloc[0:34, -1].values

In [14]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
y

array(['Inspiration', 'Inspiration', 'Inspiration', 'Sad', 'Love',
       'Inspiration', 'Sad', 'Sad', 'Sad', 'Happy', 'Love', 'Inspiration',
       'Love', 'Inspiration', 'Inspiration', 'Angry', 'Inspiration',
       'Angry', 'Sad', 'Love', 'Sad', 'Inspiration', 'Sad', 'Sad', 'Love',
       'Sad', 'Love', 'Sad', 'Calm', 'Angry', 'Love', 'Inspiration',
       'Love', 'Love'], dtype=object)

Splitting the dataset into the Training set and Test set

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


# Ensemble Methods

Bagging meta-estimator

In [47]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
classifier = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.7)
classifier.fit(X_train, y_train)

BaggingClassifier(base_estimator=KNeighborsClassifier(), max_features=0.7,
                  max_samples=0.5)

Predicting the Test set results

In [48]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[['Inspiration' 'Inspiration']
 ['Sad' 'Sad']
 ['Sad' 'Sad']
 ['Love' 'Angry']
 ['Sad' 'Inspiration']
 ['Sad' 'Love']
 ['Sad' 'Inspiration']]


Making the Confusion Matrix

In [49]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[0 0 1 0]
 [0 1 0 2]
 [0 0 0 1]
 [0 0 0 2]]


0.42857142857142855

Appyling K-fold cross validation

In [50]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 3)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 25.93 %
Standard Deviation: 5.24 %


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


In [51]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 26.00 %
Standard Deviation: 9.04 %


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


In [52]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 8)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 18.75 %
Standard Deviation: 14.88 %


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=8.
  UserWarning,


Extremely Randomized Trees

In [65]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=5)
scores.mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


0.3

In [59]:

clf2 = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
scores = cross_val_score(clf2, X_train, y_train, cv=8)
scores.mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=8.
  UserWarning,


0.26041666666666663

In [64]:
clf3 = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
scores = cross_val_score(clf3, X_train, y_train, cv=5)
scores.mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


0.34

Fit an AdaBoost classifier with 100 weak learners:



In [76]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, X_train, y_train, cv=5)
scores.mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


0.14666666666666667

Gradient Tree Boosting

In [110]:
from sklearn.ensemble import GradientBoostingClassifier

clf5 = GradientBoostingClassifier(n_estimators=13, learning_rate=2.5,max_depth=1, random_state=0).fit(X_train, y_train)
clf5.score(X_test, y_test)

0.7142857142857143

Cross validation

In [103]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = clf5, X = X_train, y = y_train, cv = 7)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=7.
  UserWarning,


Accuracy: 25.00 %
Standard Deviation: 13.36 %


## Applying Grid Search to find the best model and the best parameters

In [94]:
from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators': [10,12,13,15,20,25,40,50,80,100,150], 'learning_rate': [0.4,0.5,0.6,0.8,1.0,1.5,2.0,2.5, 3.0, 4.0,5.0], 'max_depth' : [1,2,3,4,5,6,7,8,9,10,20,100]}]
grid_search = GridSearchCV(estimator = clf5,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 2,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


Best Accuracy: 37.36 %
Best Parameters: {'learning_rate': 0.4, 'max_depth': 2, 'n_estimators': 25}


HistGradientBoostingClassifier

In [115]:
from sklearn.ensemble import HistGradientBoostingClassifier

gbdt = HistGradientBoostingClassifier(categorical_features=[1,2,3,4,5,6], min_samples_leaf=1).fit(X_train, y_train)
gbdt.score(X_test, y_test)

0.7142857142857143

# Voting Classifiers

In [118]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

clf10 = LogisticRegression(random_state=1)
clf20 = RandomForestClassifier(n_estimators=50, random_state=1)
clf30 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf10), ('rf', clf20), ('gnb', clf30)],voting='hard')

for clf, label in zip([clf10, clf20, clf30, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
  scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=5)
  print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


Accuracy: 0.29 (+/- 0.08) [Logistic Regression]
Accuracy: 0.31 (+/- 0.17) [Random Forest]
Accuracy: 0.31 (+/- 0.11) [naive Bayes]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


Accuracy: 0.26 (+/- 0.09) [Ensemble]


In [126]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from itertools import product
from sklearn.ensemble import VotingClassifier

# Training classifiers
clf1 = DecisionTreeClassifier(max_depth=4)
clf2 = KNeighborsClassifier(n_neighbors=7)
clf3 = SVC(kernel='rbf', probability=True)
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2), ('svc', clf3)],voting='soft', weights=[2, 1, 2])

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)
eclf = eclf.fit(X_train, y_train)

for clf, label in zip([clf1, clf2, clf3, eclf], ['DT', 'KNN', 'SVC', 'Ensemble']):
  scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=2)
  print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.30 (+/- 0.16) [DT]
Accuracy: 0.26 (+/- 0.03) [KNN]
Accuracy: 0.33 (+/- 0.02) [SVC]
Accuracy: 0.34 (+/- 0.12) [Ensemble]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


Using the VotingClassifier with GridSearchCV

In [130]:
from sklearn.model_selection import GridSearchCV
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='soft')

params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200]}

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X_train, y_train)
grid.best_score_
grid.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


{'lr__C': 1.0, 'rf__n_estimators': 20}